In [7]:
import minsearch
import json

with open('./documents.json', 'rt') as f:
    docs_raw = json.load(f)

In [8]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(text_fields=['question',
                             'text',
                             'section'],
                keyword_fields=['course']
)

In [11]:
q = "the course has already started, can i still enroll?"
boost_dict = {
    'enroll': 2.0,
    'started': 1.0,
    'course': 0.5
}

In [12]:
index.fit(documents)

In [14]:
results = index.search(query=q,
                 filter_dict={'course': 'machine-learning-zoomcamp'},
                 boost_dict=boost_dict,
                 num_results=5)

print(results)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you can just use this link: http

In [15]:
q

'the course has already started, can i still enroll?'

In [16]:
import ollama

In [17]:
response = ollama.chat(
    model="gpt-oss:20b",
    messages=[
        {
            "role": "user",
            "content": q
        }
    ]
)

In [26]:
print(response['message']['content'])

It depends on the institution’s policy and how the particular class is running, but in most cases you **can** still join a course that has already begun. Here’s what you can do right away:

| Step | What to Do | Why It Helps |
|------|------------|--------------|
| **1. Check the Enrollment Portal** | Log into the student portal (or the LMS you use – Blackboard, Canvas, Moodle, etc.) and look for the course in the “Add/Drop” or “Register” section. | Most systems will show you if the class is still open, if you’re on a wait‑list, or if it’s already full. |
| **2. Look for a “Late Enrollment” window** | Some schools keep a “late enrollment” or “add‑on” period that extends a few days or weeks past the first day of classes. | If the window is still open, you can submit a request directly. |
| **3. Contact the Instructor** | Send a brief email: <br>`Subject: Late enrollment request – [Course Code]`<br>`Hi Professor [Last Name],<br>`<br>`I’m writing to see if it’s still possible to enroll in

In [27]:
def get_prompt_template(context: str, question: str) -> str:
    prompt_template = f"""
    You're a course teaching assistant. Your job is to help students with their 
    questions and provide explanations on various topics.
    Answer the question based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.

    CONTEXT:
    {context.strip()}
    
    QUESTION:
    {question.strip()}
    """
    return prompt_template.strip()

In [28]:
context = ""
for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\ntext: {doc['text']}\n\n"
context = context.strip()

In [29]:
print(context)

section: General course-related questions
question: The course has already started. Can I still join it?
text: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: I just joined. What should I do next? How can I access course materials?
text: Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.
Click on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp w

In [30]:
aug_prompt = get_prompt_template(context, q)

In [32]:
print(aug_prompt)

You're a course teaching assistant. Your job is to help students with their 
    questions and provide explanations on various topics.
    Answer the question based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.

    CONTEXT:
    section: General course-related questions
question: The course has already started. Can I still join it?
text: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: I just joined. What should I do next? How can I access course materials?
text: Welcome to the course! Go to the course page (http://mlzoomc

In [33]:
aug_response = ollama.chat(
    model="gpt-oss:20b",
    messages=[
        {"role": "user", "content": aug_prompt}
    ]
)

In [34]:
print(aug_response['message']['content'])

Yes, you can enroll even after the course has started. You may miss some homework submissions, but you can still participate. To earn a certificate, submit 2 out of the 3 course projects and review 3 peers’ projects by their deadlines.


In [ ]:
def search(query, filter_dict=None, boost_dict=None, num_results=5):
    results = index.search(
        query=query,
        filter_dict=filter_dict,
        boost_dict=boost_dict,
        num_results=num_results,
    )
    return results

In [48]:
query = "how can i get the certificate?"
search_results = search(query, filter_dict={'course': 'machine-learning-zoomcamp'},
                        num_results=10, boost_dict={
                            "AWS": 2,
                            "Amazon": 2,
                            "Lambda": 2
                        })

In [49]:
search_results

[{'text': 'Yes, if you finish at least 2 out of 3 projects and review 3 peers’ Projects by the deadline, you will get a certificate. This is what it looks like: link. There’s also a version without a robot: link.',
  'section': 'General course-related questions',
  'question': 'Will I get a certificate?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, it's possible. See the previous answer.",
  'section': 'General course-related questions',
  'question': 'Will I get a certificate if I missed the midterm project?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related que

In [53]:
def build_prompt(query: str, search_results: list | list[dict]) -> str:
    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\ntext: {doc['text']}\n\n"
    context = context.strip()
    
    prompt_template = f"""
You're a course teaching assistant. Your job is to help students with their 
questions and provide explanations on various topics.
Answer the question based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

CONTEXT:
{context.strip()}

QUESTION:
{query.strip()}
"""
    return prompt_template.strip()
    

In [54]:
aug_prompt = build_prompt(query, search_results)

In [55]:
print(aug_prompt)

You're a course teaching assistant. Your job is to help students with their 
questions and provide explanations on various topics.
Answer the question based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

CONTEXT:
section: General course-related questions
question: Will I get a certificate?
text: Yes, if you finish at least 2 out of 3 projects and review 3 peers’ Projects by the deadline, you will get a certificate. This is what it looks like: link. There’s also a version without a robot: link.

section: General course-related questions
question: Will I get a certificate if I missed the midterm project?
text: Yes, it's possible. See the previous answer.

section: General course-related questions
question: The course has already started. Can I still join it?
text: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificat

In [57]:
def llm(prompt: str) -> str:
    response = ollama.chat(
        model = "gpt-oss:20b",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return response["message"]["content"].strip()

print(llm(aug_prompt))

To receive a certificate you must:

1. Complete **at least two** of the three course projects (midterm, final, or another assignment).  
2. Submit each completed project **by its deadline**.  
3. Review **at least three** peer projects for each of your submissions (the reviews must also be submitted by the deadline).

If you miss the midterm project, you can still earn the certificate by completing the other two projects and the required reviews.
